In [45]:
import numpy as np
import pandas as pd
from scipy import stats
import math
from statistics import mean
import requests 
import xlsxwriter

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets_IEX import IEX_CLOUD_API_TOKEN

In [3]:
stocks.drop(135,inplace=True)
stocks.drop(219,inplace=True)
stocks.drop(467,inplace=True)
stocks.drop(484,inplace=True)

In [4]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [5]:
lst=[]
for i in range(6):
    lst.append(stocks['Ticker'][i*100:(i+1)*100])

In [6]:
query_string=[]
for i in lst:
    query_string.append(','.join(i))

In [23]:
columns = [
            'Ticker', 
            'Price',
            'One Month Price Return',
            'One Month Return Percentile',
            'Three Month Price Return',
            'Three Month Return Percentile',
            'Six Month Price Return',
            'Six Month Return Percentile',
            'One Year Price Return', 
            'One Year Return Percentile',            
            'HQM Score',
            'Number of Shares to Buy'
            ]

In [24]:
momentum=pd.DataFrame(columns = columns)

In [25]:
for query in query_string:
    batch_url=f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={query}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_url).json()
    for ticker in query.split(','):
        momentum = momentum.append(pd.Series([ticker,
                                              data[ticker]['quote']['latestPrice'],
                                              data[ticker]['stats']['month1ChangePercent'],
                                              'N/A',
                                              data[ticker]['stats']['month3ChangePercent'],
                                              'N/A',
                                              data[ticker]['stats']['month6ChangePercent'],
                                              'N/A',
                                              data[ticker]['stats']['year1ChangePercent'],
                                              'N/A',
                                              'N/A',
                                              'N/A'], 
                                            index = columns), 
                                            ignore_index = True)

In [26]:
momentum

,Ticker,Price,One Month Price Return,One Month Return Percentile,Three Month Price Return,Three Month Return Percentile,Six Month Price Return,Six Month Return Percentile,One Year Price Return,One Year Return Percentile,HQM Score,Number of Shares to Buy
0,A,126.24,0.0233953,N/A,-0.0629542,N/A,-0.158974,N/A,-0.18143,N/A,N/A,N/A
1,AAL,14.03,-0.089737,N/A,-0.21667,N/A,-0.302698,N/A,-0.369641,N/A,N/A,N/A
2,AAP,189.03,0.0227925,N/A,-0.172491,N/A,-0.204059,N/A,-0.132486,N/A,N/A,N/A
3,AAPL,150.99,0.0580731,N/A,-0.128746,N/A,-0.17377,N/A,0.00411401,N/A,N/A,N/A
4,ABBV,157.53,0.0706508,N/A,-0.092881,N/A,0.143904,N/A,0.38178,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,120.85,0.019156,N/A,-0.0104326,N/A,-0.103602,N/A,-9.89156e-05,N/A,N/A,N/A
497,ZBH,106.30,-0.0897765,N/A,-0.186639,N/A,-0.194545,N/A,-0.353917,N/A,N/A,N/A
498,ZBRA,292.10,-0.0452195,N/A,-0.29898,N/A,-0.470816,N/A,-0.478668,N/A,N/A,N/A
499,ZION,52.87,-0.0452279,N/A,-0.221823,N/A,-0.293478,N/A,-0.0364075,N/A,N/A,N/A


In [27]:
time= [
        'One Month',
        'Three Month',
        'Six Month',
        'One Year',      
        ]


In [40]:
momentum.dropna(subset=[
        'One Month Price Return',
        'Three Month Price Return',
        'Six Month Price Return',
        'One Year Price Return',      
        ],inplace=True)

In [43]:
for index in momentum.index:
    for period in time:
        momentum.loc[index,f'{period} Return Percentile']=stats.percentileofscore(momentum[f'{period} Price Return'],momentum.loc[index,f'{period} Price Return'])

In [44]:
momentum

,Ticker,Price,One Month Price Return,One Month Return Percentile,Three Month Price Return,Three Month Return Percentile,Six Month Price Return,Six Month Return Percentile,One Year Price Return,One Year Return Percentile,HQM Score,Number of Shares to Buy
0,A,126.24,0.0233953,74.6479,-0.0629542,71.0262,-0.158974,51.9115,-0.18143,34.4064,N/A,N/A
1,AAL,14.03,-0.089737,17.7062,-0.21667,17.1026,-0.302698,20.5231,-0.369641,10.8652,N/A,N/A
2,AAP,189.03,0.0227925,74.4467,-0.172491,28.7726,-0.204059,40.6439,-0.132486,44.4668,N/A,N/A
3,AAPL,150.99,0.0580731,87.5252,-0.128746,44.8692,-0.17377,47.4849,0.00411401,67.6056,N/A,N/A
4,ABBV,157.53,0.0706508,91.7505,-0.092881,58.7525,0.143904,94.5674,0.38178,95.5734,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,120.85,0.019156,73.2394,-0.0104326,82.8974,-0.103602,61.167,-9.89156e-05,64.9899,N/A,N/A
497,ZBH,106.30,-0.0897765,17.505,-0.186639,23.9437,-0.194545,43.2596,-0.353917,12.0724,N/A,N/A
498,ZBRA,292.10,-0.0452195,36.6197,-0.29898,5.4326,-0.470816,3.21932,-0.478668,4.42656,N/A,N/A
499,ZION,52.87,-0.0452279,36.4185,-0.221823,16.0966,-0.293478,21.5292,-0.0364075,60.3622,N/A,N/A


In [50]:
for index in momentum.index:
    percentiles = []
    for period in time:
        percentiles.append(momentum.loc[index, f'{period} Return Percentile'])
    momentum.loc[index, 'HQM Score'] = mean(percentiles)

In [51]:
momentum

,Ticker,Price,One Month Price Return,One Month Return Percentile,Three Month Price Return,Three Month Return Percentile,Six Month Price Return,Six Month Return Percentile,One Year Price Return,One Year Return Percentile,HQM Score,Number of Shares to Buy
0,A,126.24,0.0233953,74.6479,-0.0629542,71.0262,-0.158974,51.9115,-0.18143,34.4064,57.998,N/A
1,AAL,14.03,-0.089737,17.7062,-0.21667,17.1026,-0.302698,20.5231,-0.369641,10.8652,16.5493,N/A
2,AAP,189.03,0.0227925,74.4467,-0.172491,28.7726,-0.204059,40.6439,-0.132486,44.4668,47.0825,N/A
3,AAPL,150.99,0.0580731,87.5252,-0.128746,44.8692,-0.17377,47.4849,0.00411401,67.6056,61.8712,N/A
4,ABBV,157.53,0.0706508,91.7505,-0.092881,58.7525,0.143904,94.5674,0.38178,95.5734,85.161,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,120.85,0.019156,73.2394,-0.0104326,82.8974,-0.103602,61.167,-9.89156e-05,64.9899,70.5734,N/A
497,ZBH,106.30,-0.0897765,17.505,-0.186639,23.9437,-0.194545,43.2596,-0.353917,12.0724,24.1952,N/A
498,ZBRA,292.10,-0.0452195,36.6197,-0.29898,5.4326,-0.470816,3.21932,-0.478668,4.42656,12.4245,N/A
499,ZION,52.87,-0.0452279,36.4185,-0.221823,16.0966,-0.293478,21.5292,-0.0364075,60.3622,33.6016,N/A


In [56]:
momentum.sort_values(by = 'HQM Score', ascending = False,inplace=True)

In [59]:
Top50=momentum[:50]

In [80]:
Top50.reset_index(drop = True, inplace = True)

In [81]:
Top50

,Ticker,Price,One Month Price Return,One Month Return Percentile,Three Month Price Return,Three Month Return Percentile,Six Month Price Return,Six Month Return Percentile,One Year Price Return,One Year Return Percentile,HQM Score,Number of Shares to Buy
0,LB,82.48,0.0798121,93.3602,0.217166,99.3964,0.81825,100,2.29165,100,98.1891,242
1,COG,22.60,0.253877,100,0.260824,99.5976,0.183,96.1771,0.353898,94.5674,97.5855,884
2,VRTX,308.30,0.156904,99.5976,0.0462617,93.9638,0.312502,98.994,0.502631,97.5855,97.5352,64
3,LLY,344.04,0.119284,98.994,0.0754274,95.7746,0.274801,98.5915,0.444693,96.1771,97.3843,58
4,GIS,78.32,0.117404,98.7928,0.0827303,96.9819,0.126014,93.159,0.322861,94.165,95.7746,255
5,CI,280.88,0.108702,98.1891,0.107835,97.5855,0.184719,96.3783,0.21038,89.9396,95.5231,71
6,MRK,95.30,0.0933193,96.3783,0.10011,97.3843,0.181133,95.9759,0.255817,92.1529,95.4728,209
7,HRB,38.00,0.0429545,82.6962,0.383108,100,0.678833,99.5976,0.62072,98.3903,95.171,526
8,MCK,332.47,0.0506906,86.3179,0.0256654,90.9457,0.296602,98.7928,0.763417,99.3964,93.8632,60
9,DLTR,170.15,0.065845,90.7445,0.0042161,87.7264,0.218553,97.3843,0.678804,98.7928,93.662,117


In [58]:
while True:
    try:
        portfolio= float(input('Enter the size of the Portfolio in USD '))
        break
    except ValueError:
        print("Please enter a Number")

Enter the size of the Portfolio in USD 1000000


In [61]:
position_size=portfolio/len(Top50.index)

In [64]:
for i in Top50.index:
    Top50.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / Top50['Price'][i])

C:\Users\Vighnesh Deshpande\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [65]:
Top50

,Ticker,Price,One Month Price Return,One Month Return Percentile,Three Month Price Return,Three Month Return Percentile,Six Month Price Return,Six Month Return Percentile,One Year Price Return,One Year Return Percentile,HQM Score,Number of Shares to Buy
273,LB,82.48,0.0798121,93.3602,0.217166,99.3964,0.81825,100,2.29165,100,98.1891,242
107,COG,22.60,0.253877,100,0.260824,99.5976,0.183,96.1771,0.353898,94.5674,97.5855,884
470,VRTX,308.30,0.156904,99.5976,0.0462617,93.9638,0.312502,98.994,0.502631,97.5855,97.5352,64
281,LLY,344.04,0.119284,98.994,0.0754274,95.7746,0.274801,98.5915,0.444693,96.1771,97.3843,58
199,GIS,78.32,0.117404,98.7928,0.0827303,96.9819,0.126014,93.159,0.322861,94.165,95.7746,255
94,CI,280.88,0.108702,98.1891,0.107835,97.5855,0.184719,96.3783,0.21038,89.9396,95.5231,71
314,MRK,95.30,0.0933193,96.3783,0.10011,97.3843,0.181133,95.9759,0.255817,92.1529,95.4728,209
225,HRB,38.00,0.0429545,82.6962,0.383108,100,0.678833,99.5976,0.62072,98.3903,95.171,526
298,MCK,332.47,0.0506906,86.3179,0.0256654,90.9457,0.296602,98.7928,0.763417,99.3964,93.8632,60
138,DLTR,170.15,0.065845,90.7445,0.0042161,87.7264,0.218553,97.3843,0.678804,98.7928,93.662,117


In [75]:
writer = pd.ExcelWriter('Quantitative_Momentum_Strategy.xlsx', engine='xlsxwriter')
Top50.to_excel(writer, sheet_name='Quantitative Momentum Strategy', index = False)

In [76]:
background_color = '#ffffff'
font_color = '#000000'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
percent_format = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [77]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],                    
                    'C': ['One-Month Price Return', percent_format],
                    'D': ['One-Month Return Percentile', percent_format],
                    'E': ['Three-Month Price Return', percent_format],
                    'F': ['Three-Month Return Percentile', percent_format],
                    'G': ['Six-Month Price Return', percent_format],
                    'H': ['Six-Month Return Percentile', percent_format],
                    'I': ['One-Year Price Return', percent_format],
                    'J': ['One-Year Return Percentile', percent_format],
                    'K': ['HQM Score', integer_format],
                    'L': ['Number of Shares to Buy', integer_format]
                    }


In [78]:
for column in column_formats.keys():
    writer.sheets['Quantitative Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Quantitative Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_format)

In [79]:
writer.save()